In [1]:
#DEBUG = True
DEBUG = False

def p(msg):
    if DEBUG:
        print(".. {}".format(msg)) 

# The InvertedList class
We implement this class which encapsulates all the important operations one can perform on an inverted list.

In [2]:
from pprint import pprint
from bisect import *

class InvertedList:
    def __init__(self, l):
        self.data = l[:] # make a copy
        self.cur = 0

    def get_list(self):
        return self.data
    
    def get_max_df(self):
        current_df = 0
        while not self.eol():          
            if self.elem() > current_df:
                current_df = self.elem()
            self.next()
        return current_df
 
    def eol(self):
        # we use cur == len(list) to indicate EOL
        return False if self.cur < len(self.data) else True
    
    def next(self, val = 1):
        # does not allow cur to be out-of-range, but use len(list) to indicate EOL
        self.cur = min(self.cur + val, len(self.data)) 
            
    def elem(self):
        if self.eol():
            return None
        else: 
            return self.data[self.cur]

    def peek(self, pos):
        if pos < len(self.data):
            return self.data[pos]
        else:
            return None
        
    def reset(self):
        self.cur = 0
    
    def dump(self):
        print("* {} : {} (cur = {})".format(self.data[:self.cur], self.data[self.cur:], self.cur))
        
    def skip_to(self, val):
        # move the cursor to the first element no smaller than `val`
        while not self.eol() and self.elem() < val:
            self.next()               

Now we implement MAX Score algorithm. Assume the idf for all terms is 1.0, and call a maxscore on each InvertedList object *within* the algorithm. The input is:
- a list of InvertedList objects
- k: which specifies how many top results we want.

In [11]:
from heapq import heappush, heappop
def Max_score(RTList,k):
    ## Required posting list(RTList),only retrieve the documents in the posting lists of RTList
    u_list = []
    PTList = RTList
    top_k = []
    ## Initialize the top_k heap with (-1, -1), the structure in top_k is (doc_id, score)
    initial_value = ()
    heappush(top_k, (-1,-1))
    print(top_k)
    current_score = []
    for list in RTList:
        u_list.append(list.get_max_df())
        list.reset()
    print(u_list)
    Flag = True
    i = 1
    ## continue scan the invertedlist until exhausting all documents in L1(L1 has the minimal amount of documents)
    while Flag == True:
        doc_id = 0       
        if u_list[-i] < top_k[0][0]:
            RTList = RTList[:-1]
            i += 1
        for obj in PTList:
            doc_id = obj.cur
            current_score.append(obj.elem())
            print(obj.elem())
            obj.next()
            if obj.eol():
                Flag = False
        print('Current_score is {}'.format(current_score))
        print(len(top_k),k,doc_id)
        ## while the top_k list is not full, just append current score directly
        if len(top_k) < k:
            current_doc_score = sum(current_score)
            heappush(top_k,(current_doc_score,doc_id))
            current_score = []
            print('The top{} results are {}'.format(k,top_k))
            continue
        ## while the top_k list is full, compare the top_k score with current score and excute corresponding algorithm
        elif len(top_k) == k:
            if sum(current_score) > top_k[0][0]:
                heappop(top_k)
                current_doc_score = sum(current_score)
                heappush(top_k,(current_doc_score,doc_id))
                current_score = []
                print('The top{} results are {}'.format(k,top_k))
                continue
        current_score = []
        print('The top{} results are {}'.format(k,top_k))

Test MAX algorithm

In [16]:
a = InvertedList([2, 8, 0, 2])                        ## u_a = 8 
b = InvertedList([1, 0, 0, 4, 0, 0, 0, 0, 0, 1, 4])   ## u_b = 4
c = InvertedList([0, 1, 2, 1, 0, 0, 0, 0, 0, 2, 2])   ## u_c = 2
inverted_list_obj = [a,b,c]
Max_score(inverted_list_obj,2)

[(-1, -1)]
[8, 4, 2]
2
1
0
Current_score is [2, 1, 0]
1 2 0
The top2 results are [(-1, -1), (3, 0)]
8
0
1
Current_score is [8, 0, 1]
2 2 1
The top2 results are [(3, 0), (9, 1)]
0
0
2
Current_score is [0, 0, 2]
2 2 2
The top2 results are [(3, 0), (9, 1)]
2
4
1
Current_score is [2, 4, 1]
2 2 3
The top2 results are [(7, 3), (9, 1)]
